# dataset link
https://www.kaggle.com/datasets/andrewmvd/animal-faces


In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
"""
loop through images and labels then create dataframe
"""

image_path = []
labels = []
data_folder = 'afhq'

for i in os.listdir(data_folder):
    for label in os.listdir(f"{data_folder}/{i}"):
        for image in os.listdir(f"{data_folder}/{i}/{label}"):
            image_path.append(f"{data_folder}/{i}/{label}/{image}")
            labels.append(label)

df = pd.DataFrame(zip(image_path, labels), columns=["path", "label"])

In [4]:
df.head()

,path,label
0,afhq/val/cat/pixabay_cat_000440.jpg,cat
1,afhq/val/cat/pixabay_cat_000099.jpg,cat
2,afhq/val/cat/pixabay_cat_000531.jpg,cat
3,afhq/val/cat/pixabay_cat_002156.jpg,cat
4,afhq/val/cat/flickr_cat_000802.jpg,cat


In [6]:
df["label"].unique()

array(['cat', 'dog', 'wild'], dtype=object)

In [5]:
"""
split data for training, testing and validation
"""

train = df.sample(frac=0.7)
test = df.drop(train.index)
val = test.sample(frac=0.5)
test = test.drop(val.index)

print(train.shape)
print(test.shape)
print(val.shape)

(11291, 2)
(2419, 2)
(2420, 2)


In [6]:
"""
encode our labels
"""
label_encoder = LabelEncoder()
label_encoder.fit(df["label"])

LabelEncoder()

In [7]:
"""
create a transformer to process images
"""

transform = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [8]:
class CustomImageDataset(Dataset):
    """
    create a dataset class to convert data to torch dataset
    """
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = torch.tensor(
            label_encoder.transform(dataframe['label'])
        ).to(device)

    def __len__(self):
        """
        return length of dataframe
        """
        return self.dataframe.shape[0]

    def __getitem__(self, idx):
        """
        get and open image using index
        """
        label = self.labels[idx]
        img_path = self.dataframe.iloc[idx, 0]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img).to(device)
        return img, label
        

In [9]:
# create torch datasets 
train_data = CustomImageDataset(dataframe=train, transform=transform)
test_data = CustomImageDataset(dataframe=test, transform=transform)
val_data = CustomImageDataset(dataframe=val, transform=transform)

In [83]:
img, label = train_data.__getitem__(2)

In [85]:
label_encoder.inverse_transform([torch.Tensor.cpu(label)])

array(['cat'], dtype=object)

In [10]:
# define model configs
LR = 1e-4
BATCH_SIZE = 16
EPOCHS = 7


In [11]:
# load the datasets for processing
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)


In [12]:
class Net(nn.Module):
    """
    define layers and net architecture
    """
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pooling = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

        self.linear = nn.Linear((128*16*16), 128)
        self.output = nn.Linear(128, len(df['label'].unique()))
        
    def forward(self, x):
        x = self.conv1(x) # -> (32, 128, 128)
        x = self.pooling(x) # -> (32, 64, 64)
        x = self.relu(x) 

        x = self.conv2(x) # -> (64, 64, 64)
        x = self.pooling(x) # -> (64, 32, 32)
        x = self.relu(x)

        x = self.conv3(x) # -> (128, 32, 32)
        x = self.pooling(x) # -> (128, 16, 16)
        x = self.relu(x)

        x = self.flatten(x)
        x = self.linear(x)
        x = self.output(x)

        return x
            

In [13]:
model = Net().to(device)

In [17]:
from torchsummary import summary

In [18]:
summary(model, input_size=(32, 128, 128))

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            896
├─Conv2d: 1-2                            18,496
├─Conv2d: 1-3                            73,856
├─MaxPool2d: 1-4                         --
├─ReLU: 1-5                              --
├─Flatten: 1-6                           --
├─Linear: 1-7                            4,194,432
├─Linear: 1-8                            387
Total params: 4,288,067
Trainable params: 4,288,067
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            896
├─Conv2d: 1-2                            18,496
├─Conv2d: 1-3                            73,856
├─MaxPool2d: 1-4                         --
├─ReLU: 1-5                              --
├─Flatten: 1-6                           --
├─Linear: 1-7                            4,194,432
├─Linear: 1-8                            387
Total params: 4,288,067
Trainable params: 4,288,067
Non-trainable params: 0

In [14]:
# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)


In [15]:
# define loss and accuracy vars for plotting, train model

train_loss_plot = []
val_loss_plot = []
train_acc_plot = []
val_acc_plot = []

for epoch in range(EPOCHS):
    train_loss = 0
    val_loss = 0
    train_acc = 0
    val_acc = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        model_train_loss = criterion(outputs, labels)
        train_loss += model_train_loss.item()
        
        model_train_loss.backward()
        
        train_accuracy = (torch.argmax(outputs, axis=1) == labels).sum().item()
        train_acc += train_accuracy
        optimizer.step()

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            model_val_loss = criterion(outputs, labels)
            val_loss += model_val_loss.item()

            val_accuracy = (torch.argmax(outputs, axis=1) == labels).sum().item()
            val_acc += val_accuracy

    train_loss_plot.append(round(train_loss / 1000, 4))
    train_acc_plot.append(
        round(
            (train_acc / train_data.__len__()) * 100,
            4
        )
    )
    val_loss_plot.append(round(val_loss / 100, 4))
    val_acc_plot.append(
        round(
            (val_acc / val_data.__len__()) * 100,
            4
        )
    )

    print(f"""
        Epoch: {epoch+1}/{EPOCHS}, 
        Train Loss: {round(train_loss / 1000, 4)},
        Train Acc: {round((train_acc / train_data.__len__()) * 100, 4)},
        Validation Loss:{round(val_loss / 100, 4)},
        Validation Acc: {round((val_acc / val_data.__len__()) * 100, 4)}
    """)
            



        Epoch: 1/7, 
        Train Loss: 0.3333,
        Train Acc: 80.4535,
        Validation Loss:0.406,
        Validation Acc: 89.2975
    

        Epoch: 2/7, 
        Train Loss: 0.1452,
        Train Acc: 92.3479,
        Validation Loss:0.232,
        Validation Acc: 94.1322
    

        Epoch: 3/7, 
        Train Loss: 0.0981,
        Train Acc: 95.058,
        Validation Loss:0.2187,
        Validation Acc: 94.3388
    

        Epoch: 4/7, 
        Train Loss: 0.072,
        Train Acc: 96.3334,
        Validation Loss:0.1665,
        Validation Acc: 96.157
    

        Epoch: 5/7, 
        Train Loss: 0.0509,
        Train Acc: 97.4582,
        Validation Loss:0.1901,
        Validation Acc: 95.4545
    

        Epoch: 6/7, 
        Train Loss: 0.0399,
        Train Acc: 98.0073,
        Validation Loss:0.2363,
        Validation Acc: 95.0
    

        Epoch: 7/7, 
        Train Loss: 0.0309,
        Train Acc: 98.5564,
        Validation Loss:0.1718,
        Validati